In [13]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import nltk, re, string
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
wordnet_lemmatizer = WordNetLemmatizer()

def marketwatch_parser():

    def tokenize_date(s):
        pattern=r'[a-z{3}]+[\.\s]+[\d{1,2}\,\s]+[\d{4}]+'                        
        tokens=nltk.regexp_tokenize(s, pattern)
        date = tokens
        return date;
    headlines=[]

    page_url= "https://www.marketwatch.com/search?q=bitcoin&m=Keyword&rpp=500&mp=806&bd=false&bdv=&rs=false"

    while page_url!=None:
        page = requests.get(page_url) 

        if page.status_code!=200:
             page_url=None
        else:
            all_data = []

            page_url = "https://www.marketwatch.com/search?q=bitcoin&m=Keyword&rpp=500&mp=806&bd=false&bdv=&rs=false"
            page = requests.get(page_url)
            soup = BeautifulSoup(page.content, "html.parser")
            divs = soup.find_all("div", class_ = "searchresult")
            titles_dictionary = []
            for idx, div in enumerate(divs):
                titles = div.select("a")
                if titles != []:
                    title = titles[0].get_text().lower()
                    titles_dictionary.append(lemmatize(title))
            divs_dates = soup.find_all("div", class_ = "deemphasized")
            dates_list = []
            for idx, div in enumerate(divs_dates):
                date = div.get_text().lower()
                date = str(tokenize_date(date)[0])
                dates_list.append(date)
            data = zip(titles_dictionary,dates_list)  
            all_data.extend(data)

        page_url = None

        keywords = ['bitcoin','cryptocurrency','cryptocurrencies', 'crypto', 'blockchain']
        filtered_data = filter(lambda x: any(word in x[0] for word in keywords), all_data)
    return filtered_data
# data_frame = pd.DataFrame.from_records(filtered_data, columns = ["Tokenized Title", "Date"])
# data_frame.to_csv("marketwatch.csv", encoding='utf-8')